<a href="https://colab.research.google.com/github/brindaasuri/Unstructured-Data/blob/main/HW2_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 2**

**GROUP MEMBERS:**

Andy Ma

Michael Crosson

Sarah Dominguez

Brinda Asuri

Destin Blanchard

Brooks Li

In [ ]:
! pip install langdetect
! pip install googletrans==4.0.0-rc1
!python -m spacy download en_core_web_md
! pip install httpcore==0.13.3 httpx==0.18.2
!pip install langdetect googletrans==4.0.0-rc1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 12.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Using cached httpcore-0.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached httpx-0.18.2-py3-none-any.whl.metadata (44 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
 

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import csv
import numpy as np
from langdetect import detect
from googletrans import Translator
import re
from collections import defaultdict
import spacy

#Task A

In [ ]:
base_url = 'https://www.ratebeer.com/beer-ratings/'
pagination_url = base_url + '0/'

translator = Translator()

review_count = 0
total_pages = 200

reviews_data = []

for page_num in range(1, total_pages + 1):
    print(f"page {page_num} of {total_pages}")
    if page_num == 1:
        url = base_url  # First page has no 0/ suffix
    else:
        url = pagination_url + str(page_num) + '/'

    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all the review rows
        rows = soup.find_all('tr')

        # Iterate over each row and extract relevant data
        for row in rows:
            # print(row)
            review_count += 1
            beer_name = row.find('a', style="font-size:20px; font-weight:bold;").text if row.find('a', style="font-size:20px; font-weight:bold;") else ''
            brewer = ''
            brewer_div = row.find('div')
            if brewer_div:
                brewer_link = brewer_div.find('a')
                if brewer_link:
                    brewer = brewer_link.text.strip()
            location = row.find('span', class_='small').text if row.find('span', class_='small') else ''
            rating = row.find('span', class_='uas').text if row.find('span', class_='uas') else ''
            review = row.find('div', style="color:#666;").text if row.find('div', style="color:#666;") else ''

            # Check if the review is in English and translate if necessary
            if review:
                try:
                    # Detect the language of the review
                    detected_language = detect(review)

                    # Translate if the language is not English
                    if detected_language != 'en':
                        translated_review = translator.translate(review, dest='en').text
                        review = translated_review
                except Exception as e:
                    print(f"Error in translation: {e}")

            reviews_data.append([beer_name, brewer, location, rating, review])

    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

print(f"Reviews found: {review_count}")

df = pd.DataFrame(reviews_data, columns=['Beer Name', 'Brewer', 'Location', 'Rating', 'Review'])

df.to_csv('beer_reviews.csv', index=False)

In [ ]:
df
df.to_csv('beer_reviews.csv', index=False)

# Task B

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from itertools import combinations
from collections import Counter

In [ ]:
#Word Frequency Analysis
df_reviews = pd.read_csv('beer_reviews.csv')
text = df_reviews['Review'].dropna().tolist()
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text
preprocessed_reviews = [clean_text(review) for review in text]
coun_vect = CountVectorizer(stop_words = 'english')
count_matrix = coun_vect.fit_transform(preprocessed_reviews)
count_array = count_matrix.toarray()
df_word_count = pd.DataFrame(data=count_array, columns=coun_vect.get_feature_names_out())
word_frequency = df_word_count.sum().sort_values(ascending=False)
print("Most common words in reviews")
print(word_frequency.head(20))
top_attributes = word_frequency.head(5).index.tolist()
print(f"Top attributes: {top_attributes}")

Most common words in reviews
head          2227
aroma         1771
white         1633
finish        1274
light         1155
fruity        1144
clear         1107
golden        1011
dry            977
malts          919
medium         870
sweet          867
pours          859
bitterness     831
taste          782
hoppy          741
tap            710
hazy           691
frothy         690
notes          666
dtype: int64
Top attributes: ['head', 'aroma', 'white', 'finish', 'light']


In [ ]:
df_word_count.to_csv('word_count.csv', index=False)

In [ ]:
#Lift Analysis
from collections import Counter
from itertools import combinations

def find_co_occurrences(reviews, attributes):
    co_occurrences = Counter()
    individual_counts = Counter()

    for review in reviews:
        if isinstance(review, str):
            review_words = set(review.split())
            present_attributes = [attr for attr in attributes if attr in review_words]

            for attr in present_attributes:
                individual_counts[attr] += 1

            if len(present_attributes) > 1:
                for pair in combinations(present_attributes, 2):
                    co_occurrences[pair] += 1

    return co_occurrences, individual_counts

attributes = ['sweet', 'aroma', 'light', 'hops', 'citrus', 'bitter', 'fruity', 'carbonation', 'bit', 'good']

co_occurrence_counts, individual_counts = find_co_occurrences(preprocessed_reviews, attributes)

total_reviews = len(preprocessed_reviews)
lift_scores = {}

for (attr1, attr2), co_count in co_occurrence_counts.items():
    prob_attr1 = individual_counts[attr1] / total_reviews
    prob_attr2 = individual_counts[attr2] / total_reviews
    prob_both = co_count / total_reviews

    lift = prob_both / (prob_attr1 * prob_attr2)
    lift_scores[(attr1, attr2)] = lift

sorted_lift_scores = sorted(lift_scores.items(), key=lambda x: x[1], reverse=True)

print("\nTop 3 attribute pairs with the highest lift values:")
for (attr1, attr2), lift_value in sorted_lift_scores[:3]:
    print(f"{attr1} and {attr2} -> Lift: {lift_value:.2f}")



Top 3 attribute pairs with the highest lift values:
carbonation and good -> Lift: 4.02
hops and bitter -> Lift: 2.21
light and carbonation -> Lift: 1.93


For the purposes of our assignment, we will use hops, bitter, and carbonation as our attributes to connect similarities with the reviews.

# Task C

In [ ]:
attributes = list(sorted_lift_scores[1][0])
attributes.append(sorted_lift_scores[0][0][0])
attributes

['hops', 'bitter', 'carbonation']

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import numpy as np

# drop rows where the 'Review' is NaN
df_reviews = df_reviews.dropna(subset=['Review'])

# specify the customer attributes
customer_attributes = "carbonation hops bitter"

documents = [customer_attributes] + df_reviews['Review'].tolist()

count_vectorizer = CountVectorizer(stop_words='english')

sparse_matrix = count_vectorizer.fit_transform(documents)

doc_term_matrix = sparse_matrix.todense()

# calculate the total sum of all counts row-wise
total_sum = doc_term_matrix.sum(axis=1)

# normalize the document-term matrix
normalized_matrix = doc_term_matrix / total_sum

df = pd.DataFrame(normalized_matrix,
columns=count_vectorizer.get_feature_names_out(),
index=['attributes'] + [f'review_{i}' for i in range(len(normalized_matrix)-1)])

cosine_sim = cosine_similarity(df.iloc[0:1], df.iloc[1:]).flatten() # calculate cosine similarities

df_similarity = pd.DataFrame({
    'product_name': df_reviews['Beer Name'],
    'product_review': df_reviews['Review'],
    'similarity_score': cosine_sim
})

# save the results to CSV file
df_similarity.to_csv('review_similarity_scores.csv', index=False)

# # Display the first few rows of the results
df_similarity.head()


,product_name,product_review,similarity_score
0,Bridge 99 Blood Orange Hefe,12oz can from brewery. Cloudy golden pour with...,0.000000
1,Trillium / Garage Project Willamette Valley,"(Tap) cloudy, bright yellow color with a small...",0.132453
2,Discretion Jugo Nuevo,Draft at O’Malley’s sports bar. Pours hazy lig...,0.000000
3,Yellowstone Hebgen Hazy,Hazy yellow gold with thin white head\nAroma p...,0.115470
4,Burlington Beer Little Wizard,Smooth flowery IPA that isn't bitter at all! m...,0.123091


In [ ]:
df_similarity.groupby('product_name')['similarity_score'].mean().sort_values(ascending=False).reset_index().head(3)

,product_name,similarity_score
0,Craven Blonde,0.430331
1,Nottingham Garibaldi,0.428845
2,Mallassepät Kultaranta,0.390360


The top three beers that we would recommend using the bag-of-words approach before calculating cosine similarities are **Craven Blonde, Nottingham Garibaldi**, and **Mallassepät Kultaranta**.

# Task D

###Vader sentiment analysis with redefining word sentiment values

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#initializing the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

#redefine default sentiment values for some words that could be considered controversial
new_words = {
    'stale': -1.5,   #increase negative sentiment
    'yeasty': -2.5,  #increase negative sentiment
    'crisp': 2.0,    #increase positive sentiment
    'refreshing': 2.0,  #increase positive sentiment
    'sour': -0.5,    #increase negative sentiment
    'meh': -0.5,     #increase negative sentiment
    'bland': -1.5,   #increase negative sentiment
    'bitter': 0.5,   #increase positive sentiment
    'grainy': 1.5,   #increase positive sentiment
    'refined': 2.5,  #increase positive sentiment
}
analyzer.lexicon.update(new_words)

#perform sentiment analysis using VADER
def perform_sentiment_analysis(df, review_column):

    #function to extract the sentiment scores from each review
    def get_sentiment(review):
        if not review:  # Handle empty reviews
            return None
        return analyzer.polarity_scores(review)

    #apply sentiment analysis to each review and store results
    df['sentiment'] = df[review_column].apply(get_sentiment)

    #extracting compound score and other metrics into separate columns, not needed really because we are focusing on
    #the compound sentiment score, specifically the updated compound score after refining words but this is useful for
    #visualization
    df['sentiment_compound'] = df['sentiment'].apply(lambda score_dict: score_dict['compound'] if score_dict else None)
    df['sentiment_pos'] = df['sentiment'].apply(lambda score_dict: score_dict['pos'] if score_dict else None)
    df['sentiment_neu'] = df['sentiment'].apply(lambda score_dict: score_dict['neu'] if score_dict else None)
    df['sentiment_neg'] = df['sentiment'].apply(lambda score_dict: score_dict['neg'] if score_dict else None)

    #dropping the 'sentiment' dictionary column if not needed
    df.drop(columns=['sentiment'], inplace=True)

    return df

#sentiment analysis with the updated lexicon
df_reviews = perform_sentiment_analysis(df_reviews, 'Review')

#DataFrame with sentiment analysis results
df_reviews.head()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Beer Name,Brewer,Location,Rating,Review,sentiment_compound,sentiment_pos,sentiment_neu,sentiment_neg,evaluation_score
0,Bridge 99 Blood Orange Hefe,Bridge 99 Brewery,"Bend, Oregon",3.1,12oz can from brewery. Cloudy golden pour with...,0.4215,0.068,0.932,0.000,0.000000
1,Trillium / Garage Project Willamette Valley,Trillium Brewing Company,"Boston, Massachusetts",4.1,"(Tap) cloudy, bright yellow color with a small...",0.5267,0.180,0.820,0.000,0.069763
2,Discretion Jugo Nuevo,Discretion Brewing,"Soquel, California",2.9,Draft at O’Malley’s sports bar. Pours hazy lig...,-0.3400,0.000,0.924,0.076,-0.000000
3,Yellowstone Hebgen Hazy,Yellowstone Beer Company,"West Yellowstone, Montana ...",3.5,Hazy yellow gold with thin white head\nAroma p...,-0.2960,0.048,0.865,0.087,-0.034179
4,Burlington Beer Little Wizard,Burlington Beer Company,"Williston, Vermont",3.6,Smooth flowery IPA that isn't bitter at all! m...,0.4866,0.102,0.870,0.028,0.059896


We updated VADER's word scores because certain words, like "crisp" or "stale," may have specific meanings in the context we're working with, such as beer reviews. By customizing these word scores, we make the sentiment analysis more accurate for our data. The updated sentiment compound is what's reflecting using these redefined words. Then, we calculate and add sentiment scores (positive, negative, neutral, and overall) for each review in the dataset. Though we are focusing on the sentiment compound, it is still good to be able to visualize what the positive, neutral and negative sentiments are.

# Task E

In [ ]:
# Merging the product and beer name dataframes
df_eval = df_reviews[['Beer Name', 'sentiment_compound']].copy()
df_eval['similarity_score'] = df_similarity['similarity_score']

# Calculate the mean similarity and sentiment scores grouped by Beer Name
grouped_scores = df_eval.groupby('Beer Name').agg({
    'sentiment_compound': 'mean',
    'similarity_score': 'mean'
}).reset_index()

# Create evaluation scores based on grouped similarity and sentiment scores
grouped_scores['evaluation_score'] = grouped_scores['similarity_score'] * grouped_scores['sentiment_compound']

# Find the top beers by sorting evaluation score in descending order
top_beers = grouped_scores.sort_values(by='evaluation_score', ascending=False)

# Print the top beers and their associated scores
top_beers[['Beer Name', 'similarity_score', 'sentiment_compound', 'evaluation_score']].head(10)


,Beer Name,similarity_score,sentiment_compound,evaluation_score
477,Four Winds Rainshadow,0.360844,0.9144,0.329956
474,Four Winds Centennial,0.342594,0.9603,0.328993
158,Bert's Juke Joint,0.321634,0.9607,0.308994
956,Pretty Decent Request Ottline,0.329914,0.9112,0.300618
841,Nottingham Garibaldi,0.428845,0.7003,0.300320
997,Right Proper Old World Charm,0.339683,0.8020,0.272426
307,Craven Blonde,0.430331,0.6249,0.268914
712,Lervig Hoppy Joe,0.268462,0.9776,0.262449
1101,Sommar Respect,0.286446,0.9154,0.262213
145,Beer'd Dogs and Boats,0.301511,0.8591,0.259028


We ranked the evaluation scores in descending order above. Based on the top evaluation scores, we would recommend **Four Winds Raishadow, Four Winds Centennial**, and **Bert's Juke Joint**. Having the higher evaluation scores mean that consumers have positive sentiments toward these beer products and therefore future consumers may also like these brands. Not to mention, a higher evaluation score also indicates a higher degree of similarity to the attributes for each product, which broadly fit the consumers' preferences.

# Task F

In [ ]:
# Here we calculate the similarity between the same docs using word vectors (spacy)
nlp = spacy.load('en_core_web_md')

# Drop rows where the 'Review' is NaN
df_reviews = df_reviews.dropna(subset=['Review'])

# Specify the customer attributes
customer_attributes = " ".join(attributes)

# Apply the nlp() function to the customer attributes and reviews
doc_customer_attributes = nlp(customer_attributes)
docs_reviews = [nlp(review) for review in df_reviews['Review']]

# Calculate similarity between the customer attributes and each review
similarity_scores = [doc_customer_attributes.similarity(doc_review) for doc_review in docs_reviews]

# Create a DataFrame to store the product name, review, and similarity score
df_similarity_vec = pd.DataFrame({
    'product_name': df_reviews['Beer Name'],
    'product_review': df_reviews['Review'],
    'rating': df_reviews['Rating'],
    'similarity_score': similarity_scores,
    'sentiment_score': df_eval['sentiment_compound']
})

df_similarity_vec

,product_name,product_review,rating,similarity_score,sentiment_score
0,Bridge 99 Blood Orange Hefe,12oz can from brewery. Cloudy golden pour with...,3.1,0.629921,0.4215
1,Trillium / Garage Project Willamette Valley,"(Tap) cloudy, bright yellow color with a small...",4.1,0.548653,0.5267
2,Discretion Jugo Nuevo,Draft at O’Malley’s sports bar. Pours hazy lig...,2.9,0.550844,-0.3400
3,Yellowstone Hebgen Hazy,Hazy yellow gold with thin white head\nAroma p...,3.5,0.677975,-0.2960
4,Burlington Beer Little Wizard,Smooth flowery IPA that isn't bitter at all! m...,3.6,0.568659,0.4866
...,...,...,...,...,...
3179,Kent Wai-iti,Clear and golden with a frothy white head and ...,3.4,0.631326,0.7845
3180,Allendale Wolf,"Cask at Newcastle Tap, Newcastle. Clear coppe...",2.8,0.588630,0.6808
3181,Fierce Pine Ridge,Can from Aldi. It pours clear amber with a med...,3.6,0.601348,0.7184
3182,Rigg & Furrow Hooves,"Cask at Enigma, North Shields. Clear mid gold...",2.4,0.534485,0.8074


In [ ]:
# calculate mean rating, similarity score, and sentiment score grouped by product_name
aggregated_df = df_similarity_vec.groupby('product_name').agg({
    'rating': 'mean',
    'similarity_score': 'mean',
    'sentiment_score': 'mean'
}).reset_index()

# get the top three product names based on aggregated similarity scores
top_three_vector_bev = aggregated_df.sort_values('similarity_score', ascending=False).head(3)['product_name'].tolist()

# filter the df to include only the top three products
top_three_vec_df = df_similarity_vec[df_similarity_vec['product_name'].isin(top_three_vector_bev)].copy()

# calculate evaluation score by multiplying **grouped** similarity and sentiment scores
top_three_vec_df['evaluation_score'] = top_three_vec_df['similarity_score'] * top_three_vec_df['sentiment_score']

top_three_vec_df


,product_name,product_review,rating,similarity_score,sentiment_score,evaluation_score
740,White Dog / Tablerock Hopzilla,Clear dark amber with lacy beige head\nAroma i...,3.7,0.837368,0.7269,0.608683
826,10 Barrel My Retirement Grease,Clear dark brown with orange highlights and ri...,4.0,0.821685,0.5859,0.481425
1234,Barbarian Cry of the Blackbirds,Cloudy deep burgundy with thin lacy pink head\...,3.4,0.841628,0.0000,0.000000


The top three recommended beers using the SpaCy word vector approach are the following: **White Dog / Tablerock Hopzilla, 10 Barrel My Retirement Grease**, and **Barbarian Cry of the Blackbirds**.

In [ ]:
# Define the customer attributes
customer_attributes = attributes

# define a function to calculate the attribute percentage
def calculate_attribute_percentage(df, attribute):
    # series here is a Series of reviews; we check for the attribute in each review
    attribute_count = df.str.contains(attribute, case=False, na=False).sum()
    # calculate the percentage based on the total number of reviews in the series
    return (attribute_count / len(df)) * 100

# group by 'product_name' and calculate the attribute percentages using the 'agg' method
grouped_vec_df = (
    top_three_vec_df.groupby('product_name')
    .agg(
        hops_percentage=('product_review', lambda x: calculate_attribute_percentage(x, customer_attributes[0])),
        bitter_percentage=('product_review', lambda x: calculate_attribute_percentage(x, customer_attributes[1])),
        carbonation_percentage=('product_review', lambda x: calculate_attribute_percentage(x, customer_attributes[2]))
    )
    .reset_index()
)

# get the top three product names based on similarity scores (b-o-w)
top_three_bow_bev = df_similarity.sort_values(by='similarity_score', ascending=False).head(3)['product_name'].tolist()

top_three_bow_df = df_similarity[df_similarity['product_name'].isin(top_three_bow_bev)]

# group by product name for b-o-w approach
grouped_bow_df = (
    top_three_bow_df.groupby('product_name')
    .agg(
        hops_percentage=('product_review', lambda x: calculate_attribute_percentage(x, customer_attributes[0])),
        bitter_percentage=('product_review', lambda x: calculate_attribute_percentage(x, customer_attributes[1])),
        carbonation_percentage=('product_review', lambda x: calculate_attribute_percentage(x, customer_attributes[2]))
    )
    .reset_index()
)

grouped_vec_df

,product_name,hops_percentage,bitter_percentage,carbonation_percentage
0,10 Barrel My Retirement Grease,0.0,0.0,0.0
1,Barbarian Cry of the Blackbirds,0.0,0.0,0.0
2,White Dog / Tablerock Hopzilla,0.0,100.0,0.0


In [ ]:
grouped_bow_df

,product_name,hops_percentage,bitter_percentage,carbonation_percentage
0,Craven Blonde,100.0,100.0,100.0
1,Mallassepät Kultaranta,100.0,100.0,0.0
2,Nottingham Garibaldi,100.0,100.0,100.0


Our recommendations inherently change when basing our beers off the vector approach versus the bag-of-words approach. The two tables above demonstrate the difference observed when using either method. The first table, based off the cosine similarities of the review and attributes vectors, shows the percentage/proportional occurrence of exact attributes in reviews. One caveat to note is that our data was based off most recent 3,000 reviews, which led to few duplicate beer names. As a result, the percentage values are not as 'granular' or precise.

We see that in the first table of captured attribute proportions across reviews grouped by beer name, a positive value of 100% only shows up for the bitter attribute for White Dog/Tablerock Hopzilla. Meanwhile, two-thirds of the beers found using the bag-of-words approach demonstrate maximum attribute capture rates. Although we learnt that the bag-of-words approach is not always better than word embeddings for recommender systems, we see that it is definitely the case here.

The difference can be observed due to how cosine similarities work behind the scenes for bag-of-words representations versus SpaCy vectors. Cosine similarities might be higher for two vectors, for example, because of 'similar' shared words like 'hops', 'hop', and 'hoppy'. On the other hand, bag-of-words cosine similarities are more tailored toward exact word matches, which in this case is being captured in our attribute capture rate tables.

# Task G


In [ ]:
# 3 highest rated products from your entire dataset and analysis
dfr = df_reviews.loc[df_reviews['Rating'] != '']
dfr['Rating'] = pd.to_numeric(dfr['Rating'], errors='coerce')
dfr = dfr.groupby('Beer Name')['Rating'].mean()
dfr = dfr.sort_values(ascending=False).head(3)
dfr = dfr.to_frame()
dfr.reset_index(inplace=True)

In [ ]:
# Update sentiment_score and rating columns in df_similarity based on df_reviews
df_similarity['sentiment_score'] = df_reviews['sentiment_compound']
df_similarity['rating'] = df_reviews['Rating']

# Calculate the average rating, similarity score, and sentiment score grouped by product_name
grouped_df = df_similarity.groupby('product_name').agg({
    'rating': 'mean',
    'similarity_score': 'mean',
    'sentiment_score': 'mean'
}).reset_index()

# Iterate through each beer name in l1 and print out the grouped values
for i in l1:
    # Filter the grouped dataframe for the current product name
    filtered_df = grouped_df[grouped_df['product_name'] == i]
    for _, row in filtered_df.iterrows():
        print(f"Product: {row['product_name']}, Rating: {row['rating']:.2f}, Similarity score: {row['similarity_score']:.4f}, Sentiment score: {row['sentiment_score']:.4f}\n")


Product: Moersleutel Eureka Chocolate Chip Cookie Dough Stout, Rating: 5.00, Similarity score: 0.1722, Sentiment score: 0.9975

Product: Olde Hickory Irish Walker 2010-, Rating: 5.00, Similarity score: 0.2200, Sentiment score: 0.5574

Product: Funky Fluid Cospell, Rating: 4.70, Similarity score: 0.0000, Sentiment score: 0.8997



In [ ]:
# check similarity scores from previous analysis for comparison
df_similarity.sort_values(by='similarity_score', ascending=False).head(3)

,product_name,product_review,similarity_score,sentiment_score,rating
166,Craven Blonde,"Cask at the Pup & Duckling, Solihull. Poured a...",0.430331,0.6249,3.3
981,Nottingham Garibaldi,Orange gold body thin lace top. Aroma of citr...,0.428845,0.7003,3.2
412,Mallassepät Kultaranta,"Draught at Juova. Golden color, white head. Bi...",0.390360,0.6240,3.3


As we can see, the top-rated beers have a far lower similarity score than the most similar beers as identified by the similarity analysis done earlier in the file. Thus we can conclude that our reccomender system based on similarity gives better recommendations than simply suggesting the top 3 beers based on rating.

Although the sentiment scores of the beers found without relying on similarity or sentiment scores are high, their corresponding cosine similarities with the preferred consumer beer attributes are quite low. The high ratings associated with those beers as compared to the ones we recommend using bag-of-words may also be due to happenstance. The takeaway is that these products would not meet the requirements of the user looking for recommendations.



# Task H

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df_reviews = df_reviews.dropna(subset=['Review'])
df_reviews['evaluation_score'] = grouped_scores['evaluation_score'] # add evaluation score to main dataframe

sampled_beers = df_reviews.sample(n=10, random_state=38).reset_index(drop=True) # random sample 10 beers

# randomly choose one beer as the target beer and move it to the top of the dataframe
np.random.seed(38)
chosen_beer_idx = np.random.choice(sampled_beers.index)
chosen_beer = sampled_beers.loc[chosen_beer_idx]
sampled_beers.drop(chosen_beer_idx, inplace=True)
sampled_beers = pd.concat([chosen_beer.to_frame().T, sampled_beers], ignore_index=True)

# Extract the name of the chosen beer for filtering purposes
chosen_beer_name = sampled_beers.loc[0, 'Beer Name']

count_vectorizer = CountVectorizer(stop_words='english')

reviews = sampled_beers['Review'].tolist()
sparse_matrix = count_vectorizer.fit_transform(reviews)

doc_term_matrix = sparse_matrix.todense()

# Calculate the total sum of all counts in the matrix (normalize the magnitudes)
total_sum = doc_term_matrix.sum(axis=1)

# Normalize the document-term matrix
normalized_matrix = doc_term_matrix / total_sum

df = pd.DataFrame(normalized_matrix,
                  columns=count_vectorizer.get_feature_names_out(),
                  index=['review_0'] + [f'review_{i+1}' for i in range(len(normalized_matrix)-1)])

# Calculate cosine similarity between the chosen beer's review (first row) and all reviews (including itself)
cosine_sim = cosine_similarity(df.iloc[0:1], df).flatten()

# wet the similarity score for all beers with the same 'Beer Name' as the chosen beer to -1 so they are not considered
for i, beer_name in enumerate(sampled_beers['Beer Name']):
    if beer_name == chosen_beer_name and i != 0:  # Exclude beers with the same name and not the chosen one itself
        cosine_sim[i] = -1

# define the attribute list
attributes = ['sweet', 'aroma', 'light', 'hops', 'citrus', 'bitter', 'fruity', 'carbonation', 'bit', 'good']

# get set of attributes present in the chosen beer's review
chosen_beer_review = reviews[0].lower()
chosen_beer_attributes = set([attr for attr in attributes if attr in chosen_beer_review])

# function to calculate attribute match proportion
def calculate_attribute_proportion(review, chosen_attributes):
    review_words = set(review.lower().split())
    matching_attributes = chosen_attributes.intersection(review_words)
    return len(matching_attributes) / len(chosen_attributes) if chosen_attributes else 0

# Calculate attribute proportion for all beers
attribute_proportions = [calculate_attribute_proportion(reviews[i], chosen_beer_attributes) for i in range(len(reviews))]

df_similarity = pd.DataFrame({
    'product_name': sampled_beers['Beer Name'],  # include the chosen beer as well
    'product_review': sampled_beers['Review'],
    'similarity_score': cosine_sim,
    'sentiment_score': sampled_beers['sentiment_compound'],
    'attribute_proportion': attribute_proportions
})

df_similarity.loc[:, 'evaluation_score'] = df_similarity['similarity_score'] * df_similarity['sentiment_score'] # calculate evaluation scores

# Group by 'product_name' and calculate the average similarity score, while excluding beers with similarity score = -1
df_similarity_filtered = df_similarity[df_similarity['similarity_score'] != -1]
df_similarity_grouped = df_similarity_filtered.groupby('product_name').agg({
    'similarity_score': 'mean',  # Calculate average similarity for each product
    'sentiment_score': 'mean',
    'evaluation_score': 'mean',
    'attribute_proportion': 'mean',  # Calculate average attribute proportion for each product
    'product_review': 'first'  # Keep the first review for reference
}).reset_index()

# Sort by similarity score to find the most similar beer, excluding the chosen one itself
most_similar_beer = df_similarity_grouped.sort_values(by='similarity_score', ascending=False).iloc[1]

# Print the most similar beer's name
print("Our randomly chosen beer:", chosen_beer_name)
print("Most similar beer (excluding those with the same name):", most_similar_beer['product_name'])


Our randomly chosen beer: Tuju Brutal Springsteen
Most similar beer (excluding those with the same name): Pentrich Heavy Metal Jacket


In [ ]:
df_similarity_grouped

,product_name,similarity_score,sentiment_score,evaluation_score,attribute_proportion,product_review
0,Howling Hops Single Hop Series No. 3 - Nelson ...,0.219687,0.2732,0.060019,0.000000,Cloudy gold with a frothy white head and a lov...
1,Lambiek Fabriek Juicy & Wild Soleille,0.263249,0.813,0.214021,0.000000,"Hazy, no head, pale dull peach colour. Downrig..."
2,Loddon Citra Quad,0.215249,0.128,0.027552,0.333333,"Cask pint at the Assembly Rooms, Epsom. Pours ..."
3,Mallinsons Science Club,0.087875,0.4767,0.04189,0.000000,"Slipped down majestically, hoppy and bitter, s..."
4,Pentrich Heavy Metal Jacket,0.355934,0.4588,0.163303,0.333333,Can from Beer Dad UK. Hazy bronze body with a ...
5,Salcombe Island Street Porter,0.210300,0.0,0.0,0.333333,"Cask pint at the Watchman, New Malden. Pours b..."
6,Skudenes Santas Candy,0.084583,0.4466,0.037775,0.333333,GYG 310824. Red color with a white head. Aroma...
7,Trilha Quartzo,0.301726,0.5106,0.154061,0.000000,On tap at Trilha Perdizes. Hazy orange with me...
8,Tuju Brutal Springsteen,1.000000,0.0,0.0,0.000000,How: Draught.\nWhere: MBCC 2024.\nAppearance: ...


The beer that we randomly chose in this case is **'Tuju Brutal Springsteen'** (**chosen one has its similarity score set equal to 1 for identification**). The beer that is most similar to it based on similarity score in additition to the total proportion of attributes that are in the randomly chosen beer captured in one of the remaining nine is **'Pentrich Heavy Metal Jacket'**.

Although we included the evaluation score as part of the analysis for this problem, it does not yield much value to our results. For example, we could have chosen **Mallisons Science Club** as the most similar beer out of the remaining nine with the one we chose. Its evaluation score, .042, is very close to the evaluation score of Tuju Brutal Springsteen, which is 0. **However,** compared to Pentrich Heavy Metal Jacket, its similarity score is about 75.5% lower and their sentiment scores are about the same. Out of these two, the better alternative is Pentrich Heavy Metal Jacket.

We also included an attribute_proportion column to determine what proportion of attributes in the randomly chosen beer are in each of the remaining nine beers.